# [Fast Implied Volatility Calculation in Python](https://stackoverflow.com/questions/61289020/fast-implied-volatility-calculation-in-python)



I am looking for a library which i can use for faster way to calculate implied volatility in python. I have options data about 1+ million rows for which i want to calculate implied volatility. what would be the fastest way i can calculate IV's. I have tried using py_vollib but it doesnt support vectorization. It takes about 5 mins approx. to calculate. Are there any other libraries which can help in faster calculation. What do people use in real time volatility calculations where there are millions of rows coming in every second?

## An example with Newton's Method

In [1]:
import numpy as np
from scipy.stats import norm
N = norm.cdf

def bs_call(S, K, T, r, vol):
    d1 = (np.log(S/K) + (r + 0.5*vol**2)*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return S * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)

def bs_vega(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

def find_vol(target_value, S, K, T, r, *args):
    MAX_ITERATIONS = 200
    PRECISION = 1.0e-5
    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = bs_call(S, K, T, r, sigma)
        vega = bs_vega(S, K, T, r, sigma)
        diff = target_value - price  # our root
        if (abs(diff) < PRECISION):
            return sigma
        sigma = sigma + diff/vega # f(x) / f'(x)
    return sigma # value wasn't found, return best guess so far

In [2]:
S = 100
K = 100
T = 11
r = 0.01
vol = 0.249

V_market = bs_call(S, K, T, r, vol)
implied_vol = find_vol(V_market, S, K, T, r)

print ('Implied vol: %.2f%%' % (implied_vol * 100))
print ('Market price = %.2f' % V_market)
print ('Model price = %.2f' % bs_call(S, K, T, r, implied_vol))

Implied vol: 24.90%
Market price = 35.83
Model price = 35.83


In [3]:
%%time
size = 10000
S = np.random.randint(100, 200, size)
K = S * 1.25
T = np.ones(size)
R = np.random.randint(0, 3, size) / 100
old_vols = np.random.randint(15, 50, size) / 100
prices = bs_call(S, K, T, R, old_vols)

params = np.vstack((prices, S, K, T, R, old_vols))
vols = list(map(find_vol, *params))

CPU times: user 15.7 s, sys: 223 ms, total: 15.9 s
Wall time: 17.2 s


In [4]:
vols[:10],old_vols[:10]

([0.4500000159209294,
  0.24000000000009808,
  0.29000000869704046,
  0.3100000007512913,
  0.24000000000161487,
  0.4500000346248506,
  0.3100000032335634,
  0.22000000001306072,
  0.20000000010857016,
  0.4700000033043562],
 array([0.45, 0.24, 0.29, 0.31, 0.24, 0.45, 0.31, 0.22, 0.2 , 0.47]))

### Optimization

If you change all calls to norm.cdf()-method into ndtr(), you will get a 2.4 time performance increase.

And if you change norm.pdf()-method into norm._pdf(), you will get another (huge) increase.

With both changes implemented, the example above dropped from 17.7 s down to 0.99 s on my machine.

You will lose error checking etc. but in this case you probably don't need all that.

See: https://github.com/scipy/scipy/issues/1914

ndtr() is in scipy.special

In [5]:
import scipy

In [6]:
new_cdf=scipy.special.ndtr

In [7]:
def bs_call(S, K, T, r, vol):
    d1 = (np.log(S/K) + (r + 0.5*vol**2)*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return S * new_cdf(d1) - np.exp(-r * T) * K * new_cdf(d2)

def bs_vega(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return S * norm._pdf(d1) * np.sqrt(T)

def find_vol(target_value, S, K, T, r, *args):
    MAX_ITERATIONS = 200
    PRECISION = 1.0e-5
    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = bs_call(S, K, T, r, sigma)
        vega = bs_vega(S, K, T, r, sigma)
        diff = target_value - price  # our root
        if (abs(diff) < PRECISION):
            return sigma
        sigma = sigma + diff/vega # f(x) / f'(x)
    return sigma # value wasn't found, return best guess so far

In [8]:
%%time
size = 10000
S = np.random.randint(100, 200, size)
K = S * 1.25
T = np.ones(size)
R = np.random.randint(0, 3, size) / 100
old_vols_x= np.random.randint(15, 50, size) / 100
prices = bs_call(S, K, T, R, old_vols)

params = np.vstack((prices, S, K, T, R, old_vols))
vols = list(map(find_vol, *params))

CPU times: user 855 ms, sys: 12.8 ms, total: 868 ms
Wall time: 979 ms


In [9]:
vols[:10],old_vols[:10]

([0.4500000159209296,
  0.24000000000161464,
  0.29000001727233676,
  0.3100000032335638,
  0.24000000000009827,
  0.4500000159209296,
  0.3100000007512911,
  0.22000000004772766,
  0.20000000138175483,
  0.4700000022471266],
 array([0.45, 0.24, 0.29, 0.31, 0.24, 0.45, 0.31, 0.22, 0.2 , 0.47]))

**Much faster!!!!**

## Python Vol lib

In [10]:
import py_vollib 
from py_vollib.black_scholes  import black_scholes as bs
from py_vollib.black_scholes.implied_volatility import implied_volatility as iv
from py_vollib.black_scholes.greeks.analytical import delta 
from py_vollib.black_scholes.greeks.analytical import gamma
from py_vollib.black_scholes.greeks.analytical import rho
from py_vollib.black_scholes.greeks.analytical import theta
from py_vollib.black_scholes.greeks.analytical import vega
import numpy as np

#py_vollib.black_scholes.implied_volatility(price, S, K, t, r, flag)

"""
price (float) – the Black-Scholes option price
S (float) – underlying asset price
sigma (float) – annualized standard deviation, or volatility
K (float) – strike price
t (float) – time to expiration in years
r (float) – risk-free interest rate
flag (str) – ‘c’ or ‘p’ for call or put.
"""
def greek_val(flag, S, K, t, r, sigma):
    price = bs(flag, S, K, t, r, sigma)
    imp_v = iv(price, S, K, t, r, flag)
    delta_calc = delta(flag, S, K, t, r, sigma)
    gamma_calc = gamma(flag, S, K, t, r, sigma)
    rho_calc = rho(flag, S, K, t, r, sigma)
    theta_calc = theta(flag, S, K, t, r, sigma)
    vega_calc = vega(flag, S, K, t, r, sigma)
    return np.array([ price, imp_v ,theta_calc, delta_calc ,rho_calc ,vega_calc ,gamma_calc])



In [11]:

S = 8400
K = 8600
sigma = 16
r = 0.07
t = 1

call=greek_val('c', S, K, t, r, sigma)

put=greek_val('p', S, K, t, r, sigma)

In [12]:
def find_iv(price,S, K, t, r,flag='c'):
    imp_v = iv(price, S, K, t, r, flag)
    return imp_v

In [13]:
call

array([ 8.40000000e+03,  1.59817715e+01, -9.09784056e-13,  1.00000000e+00,
        5.10736966e-14,  4.14642081e-13,  3.67277920e-20])

In [14]:
put

array([ 8.01858685e+03,  1.59817715e+01,  1.53781118e+00, -5.55111512e-16,
       -8.01858685e+01,  4.14642081e-13,  3.67277920e-20])

In [15]:
%%time
size = 10000
S = np.random.randint(100, 200, size)
K = S * 1.25
T = np.ones(size)
R = np.random.randint(0, 3, size) / 100
vols_x= np.random.randint(15, 50, size) / 100
prices = bs_call(S, K, T, R, old_vols)

params = np.vstack((prices, S, K, T, R))
vols = list(map(find_vol, *params))

CPU times: user 784 ms, sys: 10.6 ms, total: 794 ms
Wall time: 868 ms


In [16]:
vols[:10],old_vols[:10]

([0.4500000346248508,
  0.2400000000004174,
  0.2900000086970407,
  0.31000000075129114,
  0.240000000001615,
  0.45000001592092964,
  0.3100000007512913,
  0.22000000000326156,
  0.20000000040538482,
  0.4700000014748634],
 array([0.45, 0.24, 0.29, 0.31, 0.24, 0.45, 0.31, 0.22, 0.2 , 0.47]))